In [ ]:
import psycopg2
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from wordcloud import WordCloud
import numpy as np

# Établir une connexion à la base de données
conn = psycopg2.connect(
    host="localhost",
    database="Reddit",
    user="postgres",
    password="Simon09@09"
)

# Créer le tableau de bord interactif
def update_dashboard(subreddit):
    
    query = f"SELECT * FROM Reddit"
    data = pd.read_sql(query, conn)

    # Mise à jour des graphiques et des statistiques
    totale_messages = len(data)
    # Récupérer les données de la table "reddit" pour le subreddit sélectionné
    query = f"SELECT * FROM Reddit WHERE subreddit = '{subreddit}'"
    data = pd.read_sql(query, conn)
    # Mise à jour des graphiques et des statistiques
    total_messages = len(data)
    # Graphique à barres pour les labels
    label_counts = data['label'].value_counts()
    label_fig = px.bar(x=label_counts.index, y=label_counts.values)
    label_fig.update_layout(title_text="Répartition des labels")

    # Graphique à secteurs pour les sujets
    topic_counts = data['subreddit'].value_counts()
    topic_fig = go.Figure(data=[go.Pie(labels=topic_counts.index, values=topic_counts.values)])
    topic_fig.update_layout(title_text="Répartition des sujets")

    # Graphique en ligne pour le nombre de messages par jour de la semaine
    data['day'] = pd.Categorical(data['day'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
    data['day'] = data['day'].cat.reorder_categories(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
    day_counts = data['day'].value_counts().sort_index()
    daily_counts_fig = px.line(x=day_counts.index, y=day_counts.values)
    daily_counts_fig.update_layout(title_text="Nombre de messages par jour de la semaine")

    # Graphique en ligne pour la longueur moyenne des titres par jour de la semaine
    data['title_length'] = data['title'].str.len()
    avg_title_length = data.groupby('day')['title_length'].mean()
    avg_title_length_fig = px.line(x=avg_title_length.index, y=avg_title_length.values)
    avg_title_length_fig.update_layout(title_text="Longueur moyenne des titres par jour de la semaine")

    # Graphique en ligne pour la répartition des heures de publication
    hour_counts = data['hour'].value_counts().sort_index()
    hour_counts_fig = px.line(x=hour_counts.index, y=hour_counts.values)
    hour_counts_fig.update_layout(title_text="Répartition des heures de publication")

    # Wordcloud des titres
    title_wordcloud = WordCloud(width=400, height=300, background_color='black').generate(' '.join(data['title']))

    # Wordcloud du contenu
    content_wordcloud = WordCloud(width=400, height=300, background_color='black').generate(' '.join(data['content']))

    # Répartition de tous les subreddits
    all_subreddit_counts = pd.Series(data['subreddit'].value_counts(normalize=True))

    # Création de la figure pour afficher les graphiques dans une matrice 3x2
    fig, axs = plt.subplots(3, 2, figsize=(20, 18))

    # Affichage des graphiques dans la matrice
    axs[0, 0].bar(label_fig.data[0].x, label_fig.data[0].y)
    axs[0, 0].set_title("Répartition des labels", fontsize=16)
    axs[0, 0].set_xlabel("Labels", fontsize=14)
    axs[0, 0].set_ylabel("Nombre de messages", fontsize=14)

    # Calculate the percentage of messages for each subreddit
    subreddit_percentage = total_messages / totale_messages * 100

    # Calculer le nombre de messages pour le subreddit sélectionné
    selected_subreddit_count = topic_counts[subreddit]
    # Calculer le pourcentage de messages pour le subreddit sélectionné par rapport au total
    selected_subreddit_percentage = (total_messages/ totale_messages) * 100
    # Calculer le pourcentage des autres subreddits
    other_subreddits_percentage = 100 - selected_subreddit_percentage

    # Créer les labels pour le diagramme circulaire
    labels = [subreddit, 'Autres subreddits']
    # Créer les valeurs pour le diagramme circulaire
    values = [selected_subreddit_count, totale_messages - total_messages]

    # Créer le diagramme circulaire
    axs[0, 1].pie(values, labels=labels, autopct='%1.1f%%')
    axs[0, 1].set_title("Répartition des sujets", fontsize=16)

    axs[1, 0].plot(daily_counts_fig.data[0].x, daily_counts_fig.data[0].y)
    axs[1, 0].set_title("Nombre de messages par jour de la semaine", fontsize=16)
    axs[1, 0].set_xlabel("Jour de la semaine", fontsize=14)
    axs[1, 0].set_ylabel("Nombre de messages", fontsize=14)

    axs[1, 1].plot(avg_title_length_fig.data[0].x, avg_title_length_fig.data[0].y)
    axs[1, 1].set_title("Longueur moyenne des titres par jour de la semaine", fontsize=16)
    axs[1, 1].set_xlabel("Jour de la semaine", fontsize=14)
    axs[1, 1].set_ylabel("Longueur moyenne des titres", fontsize=14)

    axs[2, 0].plot(hour_counts_fig.data[0].x, hour_counts_fig.data[0].y)
    axs[2, 0].set_title("Répartition des heures de publication", fontsize=16)
    axs[2, 0].set_xlabel("Heure", fontsize=14)
    axs[2, 0].set_ylabel("Nombre de messages", fontsize=14)

    axs[2, 1].imshow(content_wordcloud, interpolation='bilinear')
    axs[2, 1].set_title("Wordcloud des contenu", fontsize=16)
    axs[2, 1].axis('off')

    # Réglage des espacements
    plt.tight_layout()

    # Affichage du tableau de bord
    clear_output(wait=True)
    display(HTML("<h1 style='text-align:center;color:blue;'>Tableau de bord Reddit</h1>"))
    display(HTML(f"<h3 style='text-align:center;color:black;'>Sujet sélectionné : <span style='color:red;'>{subreddit}</span></h3>"))
    display(HTML(f"<h4 style='text-align:center;color:red;'>Nombre total de messages : <span style='color:black;'>{total_messages}</span></h4>"))
    plt.show()

    # Calcul des statistiques
    avg_content_length = data['content'].str.len().mean()
    sentiment_ratio = data['label'].value_counts(normalize=True)

    # Affichage des informations statistiques
    display(HTML("<h2 style='color:blue;'>Informations statistiques</h2>"))
    display(HTML(f"<h4 style='color:black;'>Longueur moyenne des titres : <span style='color:red;'>{avg_title_length.mean():.2f}</span></h4>"))
    display(HTML(f"<h4 style='color:black;'>Longueur moyenne du contenu : <span style='color:red;'>{avg_content_length:.2f}</span></h4>"))
    display(HTML("<h4 style='color:black;'>Répartition de tous les subreddits :</h4>"))
    display(all_subreddit_counts)

# Charger les données initiales pour créer la liste déroulante
query = "SELECT DISTINCT subreddit FROM Reddit"
subreddit_options = pd.read_sql(query, conn)['subreddit'].tolist()
topic_dropdown = widgets.Dropdown(options=subreddit_options, description='Sujet:')
widgets.interactive(update_dashboard, subreddit=topic_dropdown)

interactive(children=(Dropdown(description='Sujet:', options=('Bitcoin', 'ukraine', 'france'), value='Bitcoin'…